# Neuronales Netz zur Vorhersage von Fahrtkosten

Fragestellung: Prognostizieren von Fahrtkosten: Wie teuer ist eine Fahrt, wenn ich in einer bestimmten Zone zu einer bestimmten Zeit einsteige?
Als Datenbasis für das Training werden die Taxisdaten aus dem Jahr 2019 verwendet.
In der Folge werden hierfür zwei Testfälle betrachtet. In dem einen ersten werden die Taxidaten aus dem Jahr 2019 aufgeteilt, sodass ein Teil der Daten als Testdaten verwendet werden.
In dem zweiten Fall werden die Daten aus einem zukünftigen Monat verwendet.

## Laden der Daten


Zuerst werden die Daten aus der csv-Datei geladen. Hiefür wurde vorher seitens der Gruppe eine vorbearbeitete csv Datei verwendet.

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import tensorflow as tf
from tensorflow import keras
import time
from datetime import datetime
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
# from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

input_data = pd.read_csv("../data/processed/randomized_aggregated_for_one_year.csv")
input_data


KeyboardInterrupt



## Preprocessing der Daten
Im nächsten Schritt werden die eingelesenen Daten vorbearbeitet. Wichtig ist es, dass null Werte gegen 0 ersetzt werden, auch werden die Datumswerte in Integer-Werte verändert. Zudem werden sämtliche String-Datentypen aus dem Dataframe entfernt.
Des Weiteren werden weitere Spalten angelegt, sodass die aktuelle Uhrzeit oder der aktuelle Monat mittels Integer wiedergegeben werden kann.

In [ ]:
input_data['tpep_pickup_datetime'] = pd.to_datetime(input_data['tpep_pickup_datetime'], format='%Y-%m-%d %H:%M:%S')
input_data['tpep_dropoff_datetime'] = pd.to_datetime(input_data['tpep_dropoff_datetime'], format='%Y-%m-%d %H:%M:%S')
input_data['PULocationID'].fillna(-1, inplace=True)
input_data['DOLocationID'].fillna(-1, inplace=True)
input_data['tpep_pickup_month'] = input_data['tpep_pickup_datetime'].dt.month
input_data['tpep_dropoff_month'] = input_data['tpep_dropoff_datetime'].dt.month
input_data['tpep_pickup_day_numeric'] = input_data['tpep_pickup_datetime'].dt.day
input_data['tpep_dropoff_day_numeric'] = input_data['tpep_dropoff_datetime'].dt.day
input_data['tpep_pickup_day_name'] = input_data['tpep_pickup_datetime'].dt.day_name()
input_data['tpep_dropoff_day_name'] = input_data['tpep_dropoff_datetime'].dt.day_name()
input_data['tpep_pickup_hour'] = input_data['tpep_pickup_datetime'].dt.hour
input_data['tpep_dropoff_hour'] = input_data['tpep_dropoff_datetime'].dt.hour
input_data['tpep_pickup_day'] = input_data['tpep_pickup_datetime'].dt.strftime("%w").astype(int)
input_data['tpep_dropoff_day'] = input_data['tpep_dropoff_datetime'].dt.strftime("%w").astype(int)
input_data['tpep_pickup_datetime'] = input_data['tpep_pickup_datetime'].apply(lambda x: int(x.strftime('%Y%m%d')))
input_data['tpep_dropoff_datetime'] = input_data['tpep_dropoff_datetime'].apply(lambda x: int(x.strftime('%Y%m%d')))
input_data = input_data.fillna(0)
input_data = input_data.drop(["tpep_dropoff_day_name"], axis=1)
input_data = input_data.drop(["tpep_pickup_day_name"], axis=1)
input_data = input_data.drop(["store_and_fwd_flag"], axis=1)

input_data

## Normalisierung der Daten
In diesem Abschnitt werden die Traingsdaten normalisiert. Im Anschluss werden diese in einen Trainings- sowie Testdatensatz aufgeteilt.

In [ ]:
#Definition der Features und der TargetVariable (y)
Features = ['PULocationID', 'DOLocationID', 'tpep_pickup_month', 'tpep_pickup_day', 'tpep_pickup_hour']
TargetVariable = ['total_amount']

#laden der Daten in einen seperaten Dataframe
X = input_data[Features].values
y = input_data[TargetVariable].values

# definiert StandardScaler für das normalisieren Daten
PredictorScaler = StandardScaler()
TargetVarScaler = StandardScaler()

# identifiziert das minimum und maximum Daten für die spätere Transformation
PredictorScalerFit = PredictorScaler.fit(X)
TargetVarScalerFit = TargetVarScaler.fit(y)

# transformiert die Daten (Normalisierung)
X = PredictorScalerFit.transform(X)
y = TargetVarScalerFit.transform(y)

#teilt die Daten in Test- und Traingsdaten auf (30% Testdaten, 70% Trainingsdaten)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

## Modelldefintion für das Regressionsmodell

Modell zur Vorhersage von Fahrtkosten einzelnen Fahrten. Dafür wird nur eine Output-unit verwendet. Des Weiteren wird der mean squared error als loss-function verwendet, da es sich bei dem Modell um ein Regressionsmodell handelt. Die Parameter wurden durch testen identifiziert.


In [ ]:
model = Sequential([
    Dense(units=32, input_dim=X_train.shape[1], activation='relu'),
    Dense(units=32, activation='relu'),
    Dense(units=32, activation='relu'),
    Flatten(),
    Dense(units=1)
])
model.summary()

model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mean_squared_error'])
model.fit(x=X_train, y=y_train, batch_size=100, epochs=20, shuffle=True,
          verbose=2)




In [ ]:
#Vorhersage mit dem Testdaten
Predictions = model.predict(X_test)

# die inversive Transformation der Vorhersagen, labeldaten und Testdaten
Predictions = TargetVarScalerFit.inverse_transform(Predictions)
y_test_orig = TargetVarScalerFit.inverse_transform(y_test)
Test_Data = PredictorScalerFit.inverse_transform(X_test)

#Erstellen eines neuen Dataframes mit den vorhergesagten Fahrtzeiten und den tatsächlichen Fahrtzeiten
TestingData = pd.DataFrame(data=Test_Data, columns=Features)
TestingData['total_amount'] = y_test_orig
TestingData['predicted amount'] = Predictions
TestingData.head(10)

## Zweiter Testdatensatz: Vorhersage für Fahrtkosten aus dem Jahr 2020 (Mai)
In dem diesem Abschnitt werden nun Testdaten aus dem Jahr 2020 verwendet. Für diese Testdaten soll das vorher erstellte Model nun Vorhersagen erstellen.

Im ersten Schritt wird die csv Datei geladen. Daraufhin wird ein identisches Preprocessing der Daten durchgeführt.

In [ ]:
# Einlesen einer weiteren csv aus dem nächsten Jahr
input_test_data = pd.read_csv("../data/processed/yellow_tripdata_2020-05.csv", nrows=50)

#Data preprocesssing
input_test_data['tpep_pickup_datetime'] = pd.to_datetime(input_test_data['tpep_pickup_datetime'],
                                                         format='%Y-%m-%d %H:%M:%S')
input_test_data['tpep_dropoff_datetime'] = pd.to_datetime(input_test_data['tpep_dropoff_datetime'],
                                                          format='%Y-%m-%d %H:%M:%S')
input_test_data['trip_duration'] = (
        input_test_data['tpep_dropoff_datetime'] - input_test_data['tpep_pickup_datetime']).dt.seconds
input_test_data['trip_times (min)'] = input_test_data['trip_duration'].apply(lambda x: x / 60)
input_test_data['PULocationID'].fillna(-1, inplace=True)
input_test_data['DOLocationID'].fillna(-1, inplace=True)
input_test_data['tpep_pickup_month'] = input_test_data['tpep_pickup_datetime'].dt.month
input_test_data['tpep_dropoff_month'] = input_test_data['tpep_dropoff_datetime'].dt.month
input_test_data['tpep_pickup_day_numeric'] = input_test_data['tpep_pickup_datetime'].dt.day
input_test_data['tpep_dropoff_day_numeric'] = input_test_data['tpep_dropoff_datetime'].dt.day
input_test_data['tpep_pickup_day_name'] = input_test_data['tpep_pickup_datetime'].dt.day_name()
input_test_data['tpep_dropoff_day_name'] = input_test_data['tpep_dropoff_datetime'].dt.day_name()
input_test_data['tpep_pickup_hour'] = input_test_data['tpep_pickup_datetime'].dt.hour
input_test_data['tpep_dropoff_hour'] = input_test_data['tpep_dropoff_datetime'].dt.hour
input_test_data['tpep_pickup_day'] = input_test_data['tpep_pickup_datetime'].dt.strftime("%w").astype(int)
input_test_data['tpep_dropoff_day'] = input_test_data['tpep_dropoff_datetime'].dt.strftime("%w").astype(int)
input_test_data['tpep_pickup_datetime'] = input_test_data['tpep_pickup_datetime'].apply(
    lambda x: int(x.strftime('%Y%m%d')))
input_test_data['tpep_dropoff_datetime'] = input_test_data['tpep_dropoff_datetime'].apply(
    lambda x: int(x.strftime('%Y%m%d')))
input_test_data = input_test_data.fillna(0)
input_test_data = input_test_data.drop(["tpep_dropoff_day_name"], axis=1)
input_test_data = input_test_data.drop(["tpep_pickup_day_name"], axis=1)
input_test_data = input_test_data.drop(["store_and_fwd_flag"], axis=1)
input_test_data



## Normalisierung der Daten
In diesem Schritt wird analog zu der vorherigen Normalisierung vorgegenagen. Nur in diesem Fall ist es nicht notwenig die labels seperat zu transformieren.

In [ ]:
#laden der Daten in einen seperaten Dataframe
X_test_features = input_test_data[Features].values
y_test_target = input_test_data[TargetVariable].values

# definiert StandardScaler für das normalisieren Daten
test_PredictorScaler = StandardScaler()
test_TargetVarScaler = StandardScaler()

# identifiziert das minimum und maximum Daten für die spätere Transformation
test_TargetVarScalerFit = test_TargetVarScaler.fit(y_test_target)
test_PredictorScaler = test_PredictorScaler.fit(X_test_features)

# Transformation der Testdaten
X_test_features = test_PredictorScaler.transform(X_test_features)





In [ ]:
# vorhersage mit den Testdaten
second_predictions = model.predict(X_test_features)

# die inversive Transformation der Vorhersagen und Testdaten
second_predictions = test_TargetVarScaler.inverse_transform(second_predictions)
second_Test_Data = test_PredictorScaler.inverse_transform(X_test_features)

#Erstellen eines neuen Dataframes mit den vorhergesagten Fahrtzeiten und den tatsächlichen Fahrtzeiten
second_TestingData = pd.DataFrame(data=second_Test_Data, columns=Features)
second_TestingData['total_amount'] = y_test_target
second_TestingData['predicted amount'] = second_predictions
second_TestingData.head(30)

## Visualisierung der Ergebnisse
Nun
werden
die
vorhergesagten
Fahrtzeiten
mit
den
tatsächlichen
Fahrtzeiten
gegenüber
gestellt

In [ ]:
plt.plot(second_TestingData['total_amount'], color='red', label='Echter Betrag')
plt.plot(second_TestingData['predicted amount'], color='blue', label='Vorhergesagter Betrag')
plt.title('Gegenüberstellung der vorhergesagten und tatsächlichen Fahrkosten')
plt.xlabel('Fahrtnummer')
plt.ylabel('Betrag (in $)')
plt.legend()
plt.figure(figsize=(1000, 1000))
plt.show()




In dem folgenden Diagramm wird die Regressionsergebnisse in einem Scatter dargestellt.

In [ ]:
#scatter für die Darstellung der Ergebnisse
sn.regplot(x='total_amount', y='predicted amount', data=second_TestingData)
